In [130]:
import pandas as pd 
import numpy as np 
import pulp 
import folium

In [131]:
data = pd.read_csv('distance_matrix.csv', index_col=0)

data

map = folium.Map(location=(23.25,77.41),zoom_start =5,max_zoom=10,min_zoom=5)
map

for i in range (50):
    cord = (data.iloc[i][50],data.iloc[i][51])
    folium.Marker(location=cord,
                  icon= folium.Icon(icon='glyphicon-map-marker',
                                     prefix='glyphicon', 
                                     icon_size=(5,5), 
                                     icon_color='red',shadow_size=(0,0)),
                    popup=data.index[i]).add_to(map)

map

/tmp/ipykernel_44729/2051879957.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cord = (data.iloc[i][50],data.iloc[i][51])


In [132]:
# Define the problem
prob = pulp.LpProblem("Facility Location", pulp.LpMinimize)

# Parameters
num_faci = 50
num_dema = 50

/home/delhivery/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [133]:
x = pulp.LpVariable.dicts('X', range(num_faci), cat='Binary')
y = pulp.LpVariable.dicts('Y', (range(num_dema), range(num_faci)), cat='Binary')


In [134]:
# Objective function
prob += pulp.lpSum(y[demand][facility] * data.iloc[facility, demand]
                   for demand in range(num_dema)
                   for facility in range(num_faci))


In [135]:
#Total number of facilities constraint = 3
prob += pulp.lpSum(x[facility] for facility in range(num_faci)) == 10, 'Total_Facilities'

# Every demand zone is connected to one facility only
for demand in range(num_dema):
    prob += pulp.lpSum(y[demand][facility] for facility in range(num_faci)) == 1, f'demand_{demand}'

# Contingency Condition: yij <= xi
for demand in range(num_dema):
    for facility in range(num_faci):
        prob += y[demand][facility] <= x[facility], f'Contingency_{demand}_{facility}'

In [136]:
# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/delhivery/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8daf08bfe9a64bbc97505f6ad0a3c834-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/8daf08bfe9a64bbc97505f6ad0a3c834-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2556 COLUMNS
At line 17657 RHS
At line 20209 BOUNDS
At line 22760 ENDATA
Problem MODEL has 2551 rows, 2550 columns and 7550 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7938.63 - 0.05 seconds
Cgl0004I processed model has 2551 rows, 2550 columns (2550 integer (2550 of which binary)) and 7550 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 7938.63
Cbc0038I Before mini branch and bound, 2550 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improv

1

In [137]:
# Print the status
print("Status:", pulp.LpStatus[prob.status])

# Print the results
for facility in range(num_faci):
    if pulp.value(x[facility])== 1:
        print(f'Facility {data.index[facility]} is open')
        #print(f'X[{facility}] =', pulp.value(x[facility]))
    
for demand in range(num_dema):
    for facility in range(num_faci):
        if pulp.value(y[demand][facility])==1:
            print(f'{data.index[facility]} is supplying to {data.index[demand]}')
            #print(f'Y[{demand}][{facility}] =', pulp.value(y[demand][facility]))

Status: Optimal
Facility Ahmedabad is open
Facility Pune is open
Facility Faridabad is open
Facility Dhanbad is open
Facility Allahabad is open
Facility Coimbatore is open
Facility Jabalpur is open
Facility Vijayawada is open
Facility Guwahati is open
Facility Jalandhar is open
Pune is supplying to Mumbai
Faridabad is supplying to Delhi
Coimbatore is supplying to Bengaluru
Vijayawada is supplying to Hyderabad
Ahmedabad is supplying to Ahmedabad
Vijayawada is supplying to Chennai
Dhanbad is supplying to Kolkata
Pune is supplying to Pune
Faridabad is supplying to Jaipur
Ahmedabad is supplying to Surat
Allahabad is supplying to Lucknow
Allahabad is supplying to Kanpur
Jabalpur is supplying to Nagpur
Vijayawada is supplying to Visakhapatnam
Jabalpur is supplying to Bhopal
Dhanbad is supplying to Patna
Jalandhar is supplying to Ludhiana
Faridabad is supplying to Agra
Pune is supplying to Nashik
Faridabad is supplying to Faridabad
Faridabad is supplying to Meerut
Ahmedabad is supplying to Ra

In [138]:

# Plot the route
for demand in range(num_dema):
    for facility in range(num_faci):
        if pulp.value(y[demand][facility])==1:
            waypoints = [(data.iloc[facility][50],data.iloc[facility][51]),
                         (data.iloc[demand][50],data.iloc[demand][51])]
            #print(f'{data.index[facility]} is supplying to {data.index[demand]}')
            folium.PolyLine(locations=waypoints, color='blue', weight=2, opacity=1).add_to(map)



/tmp/ipykernel_44729/1238802938.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  waypoints = [(data.iloc[facility][50],data.iloc[facility][51]),
/tmp/ipykernel_44729/1238802938.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (data.iloc[demand][50],data.iloc[demand][51])]


In [139]:
map